In [45]:
from urllib.request import urlretrieve

In [46]:
#@title 'https://indexes.nasdaqomx.com/index/Weighting/NDX'
#Excelファイルをダウンロード可能

boxurl = 'https://tus.box.com/shared/static/xf065fiet5gkcnbljynmbn5wmy4kz2m4.xlsx'
filename = 'SODWeightings_20240522_NDX.xlsx'

urlretrieve(boxurl, filename)

('SODWeightings_20240522_NDX.xlsx',
 <http.client.HTTPMessage at 0x15519079b9d0>)

In [47]:
import pandas as pd

In [48]:
!python -m pip install "pandas[excel]"


In [49]:
df_nasdaqomx = pd.read_excel(filename).drop([0,1,2,3]).set_axis(["Company Name","Security Symbol"], axis=1).reset_index(drop=True)
df_nasdaqomx

,Company Name,Security Symbol
0,APPLE INC.,AAPL
1,"AIRBNB, INC CL A CMN",ABNB
2,ADOBE INC.,ADBE
3,ANALOG DEVICES CMN,ADI
4,AUTOMATIC DATA PROCS,ADP
...,...,...
96,WALGREENS BTS ALN CM,WBA
97,WRNR BRS DS CM WI,WBD
98,WORKDAY INC CL A,WDAY
99,XCEL ENERGY CMN,XEL


In [50]:
# 取出“股票编号”列（不区分大小写寻找含 symbol 的列名更稳）
sym_col = next(c for c in df_nasdaqomx.columns if "symbol" in str(c).lower())

tickers = (
    df_nasdaqomx[sym_col]
      .dropna()                                  # 去掉空值
      .astype(str).str.strip()                   # 去前后空格
      .str.replace(r"\s+", "", regex=True)       # 去中间空白
      .str.replace(r"[^A-Za-z0-9.\-]", "", regex=True)  # 清掉奇怪符号，保留.和-
      .str.upper()
      .pipe(pd.unique)                           # 按出现顺序去重
      .tolist()
)

print("count:", len(tickers))
print(tickers)

count: 101
['AAPL', 'ABNB', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AEP', 'AMAT', 'AMD', 'AMGN', 'AMZN', 'ANSS', 'ASML', 'AVGO', 'AZN', 'BIIB', 'BKNG', 'BKR', 'CCEP', 'CDNS', 'CDW', 'CEG', 'CHTR', 'CMCSA', 'COST', 'CPRT', 'CRWD', 'CSCO', 'CSGP', 'CSX', 'CTAS', 'CTSH', 'DASH', 'DDOG', 'DLTR', 'DXCM', 'EA', 'EXC', 'FANG', 'FAST', 'FTNT', 'GEHC', 'GFS', 'GILD', 'GOOG', 'GOOGL', 'HON', 'IDXX', 'ILMN', 'INTC', 'INTU', 'ISRG', 'KDP', 'KHC', 'KLAC', 'LIN', 'LRCX', 'LULU', 'MAR', 'MCHP', 'MDB', 'MDLZ', 'MELI', 'META', 'MNST', 'MRNA', 'MRVL', 'MSFT', 'MU', 'NFLX', 'NVDA', 'NXPI', 'ODFL', 'ON', 'ORLY', 'PANW', 'PAYX', 'PCAR', 'PDD', 'PEP', 'PYPL', 'QCOM', 'REGN', 'ROP', 'ROST', 'SBUX', 'SIRI', 'SNPS', 'TEAM', 'TMUS', 'TSLA', 'TTD', 'TTWO', 'TXN', 'VRSK', 'VRTX', 'WBA', 'WBD', 'WDAY', 'XEL', 'ZS']


In [51]:
!pip install pandas

In [52]:
!pip install yfinance

In [53]:
import pandas as pd
import yfinance as yf

In [54]:
import pandas as pd, yfinance as yf

data = yf.download(tickers, start="2019-01-01", end=None, group_by='ticker', auto_adjust=True)

def has_enough(t):
    try:
        close = data[t]['Close'].dropna()
        return len(close) >= (96 + 24)   # 至少能产生一个样本
    except Exception:
        return False

kept = [t for t in tickers if has_enough(t)]
dropped = sorted(set(tickers) - set(kept))
print(f"kept={len(kept)}, dropped={len(dropped)} -> {dropped[:10]}")


[*********************100%***********************]  101 of 101 completed


kept=101, dropped=0 -> []


In [55]:
first_dates = {t: data[t]['Close'].dropna().index.min() for t in kept}
s = pd.Series(first_dates)
start_at_90pct = pd.to_datetime(s.quantile(0.90))   # 90% 覆盖
print("start for 90% coverage:", start_at_90pct.date())


start for 90% coverage: 2019-01-02


In [56]:
!pip install "gluonts[torch]"

In [57]:
from gluonts.dataset.repository.datasets import get_download_path
from pathlib import Path

In [58]:
get_download_path()

PosixPath('/home/shihongyue/.gluonts')

In [59]:
root = Path(get_download_path()) / "datasets" / "nasdaq100_prices"
(root).mkdir(parents=True, exist_ok=True)
(root / "train").mkdir(exist_ok=True)
(root / "test").mkdir(exist_ok=True)

In [60]:
import json

In [61]:
import json
import pandas as pd

# ------- 兼容取某只股票的收盘价（支持两种 MultiIndex 顺序 & 'Adj Close'）-------
def get_close_series(df: pd.DataFrame, ticker: str) -> pd.Series:
    if isinstance(df.columns, pd.MultiIndex):
        lv0 = df.columns.get_level_values(0)
        lv1 = df.columns.get_level_values(1)

        # 形如 ('Close','AAPL')
        if 'Close' in lv0:
            s = df['Close'].get(ticker)
        elif 'Adj Close' in lv0:
            s = df['Adj Close'].get(ticker)
        # 形如 ('AAPL','Close')
        elif 'Close' in lv1:
            s = df.xs('Close', axis=1, level=1).get(ticker)
        elif 'Adj Close' in lv1:
            s = df.xs('Adj Close', axis=1, level=1).get(ticker)
        else:
            s = None
    else:
        s = df.get('Close', df.get('Adj Close'))

    if s is None:
        return pd.Series(dtype='float32')

    return s.dropna().astype('float32')

# ------- 生成 train.json / test.json -------
SPLIT = 0.8                 # 训练/测试比例
CONTEXT = 96                # 和训练配置一致
PRED = 24                   # 和训练配置一致
MIN_LEN = CONTEXT + PRED    # 每条序列至少长度

(train_path := (root / "train" / "train.json").resolve()).parent.mkdir(parents=True, exist_ok=True)
(test_path  := (root / "test"  / "test.json").resolve()).parent.mkdir(parents=True, exist_ok=True)

with open(train_path, "w") as f_train, open(test_path, "w") as f_test:
    kept = 0
    for ticker in tickers:
        close = get_close_series(data, ticker)

        # 序列过短或异常，跳过
        if len(close) < MIN_LEN:
            continue

        n = len(close)
        # k 既要保证训练段 >= CONTEXT，也要保证测试段 >= PRED
        k = int(n * SPLIT)
        k = max(CONTEXT, min(k, n - PRED))
        if n - k < PRED:   # 仍不够预测窗口就跳过
            continue

        train_rec = {
            "start": str(close.index[0]),
            "target": close.iloc[:k].tolist(),
            "item_id": ticker,
        }
        test_rec = {
            "start": str(close.index[k]),
            "target": close.iloc[k:].tolist(),
            "item_id": ticker,
        }
        f_train.write(json.dumps(train_rec) + "\n")
        f_test.write(json.dumps(test_rec) + "\n")
        kept += 1

print(f"数据已保存到: {root}  | 写入条数: {kept}  (train: {train_path}, test: {test_path})")

数据已保存到: /home/shihongyue/.gluonts/datasets/nasdaq100_prices  | 写入条数: 101  (train: /home/shihongyue/.gluonts/datasets/nasdaq100_prices/train/train.json, test: /home/shihongyue/.gluonts/datasets/nasdaq100_prices/test/test.json)


In [62]:
# 写入 metadata.json
metadata = {"freq": "B","prediction_length": 24,"feat_static_cat": [],"feat_static_real": [],"feat_dynamic_real": [],"target_name": "Close"}

with (root / "metadata.json").open("w") as f:
    json.dump(metadata, f)

In [63]:
root

PosixPath('/home/shihongyue/.gluonts/datasets/nasdaq100_prices')